In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))

import pandas as pd
from pycaret.classification import ClassificationExperiment
import src.utils as utils
import src.calcEMA as calc_utils
import src.myenv as myenv
from datetime import datetime

from itertools import combinations
import plotly.express as px

<built-in method __dir__ of sqlite3.Connection object at 0x7fd583d84a40>


In [2]:
# Variables
# ETCUSDT BTCUSDT
# symbol = 'ETHUSDT'
symbol = 'BNBUSDT'
interval = '1h'
# lightgbm  xgboost lr et rf
estimator = 'knn'
_compare_models = False

start_train_date = '2010-01-01'  # train < and test >=
start_test_date = '2023-01-01'  # train < and test >=

stop_loss = 1.5
# regression_times = 0  # 24 * 30 * 2  # horas
times_regression_PnL = 24
normalize = True
use_gpu = False
tune_model = True
apply_combination_features = False

In [3]:
cols = myenv.all_klines_cols.copy()
cols.remove('ignore')
data = utils.get_data(symbol=symbol, save_database=False, interval=interval, tail=-1, columns=cols, parse_dates=True, start_date=start_train_date)
data = data[data['open_time'] >= start_train_date]
data = utils.parse_type_fields(data, parse_dates=True)
data = utils.adjust_index(data)
data = utils.truncate_data_to_train(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52161 entries, 2017-11-10 07:00:00 to 2023-10-28 16:00:00
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     52161 non-null  datetime64[ns]
 1   open                          52161 non-null  float32       
 2   high                          52161 non-null  float32       
 3   low                           52161 non-null  float32       
 4   close                         52161 non-null  float32       
 5   volume                        52161 non-null  float32       
 6   close_time                    52161 non-null  int64         
 7   quote_asset_volume            52161 non-null  float32       
 8   number_of_trades              52161 non-null  int16         
 9   taker_buy_base_asset_volume   52161 non-null  float32       
 10  taker_buy_quote_asset_volume  52161 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol
ix_open_time,,,,,,,,,,,,
2017-11-10 07:00:00,2017-11-10 07:00:00,1.948700,1.948800,1.895000,1.895000,1751.890015,1510300799999,3.368215e+03,22,308.000000,6.002211e+02,BNBUSDT
2017-11-10 08:00:00,2017-11-10 08:00:00,1.945900,1.970000,1.880000,1.970000,43483.128906,1510304399999,8.331019e+04,95,18540.289062,3.608887e+04,BNBUSDT
2017-11-10 09:00:00,2017-11-10 09:00:00,1.930100,2.010100,1.850000,1.850000,37745.929688,1510307999999,7.401212e+04,123,14075.990234,2.809465e+04,BNBUSDT
2017-11-10 10:00:00,2017-11-10 10:00:00,1.833500,1.920000,1.833500,1.890000,9118.969727,1510311599999,1.681954e+04,35,1742.750000,3.280829e+03,BNBUSDT
2017-11-10 11:00:00,2017-11-10 11:00:00,1.855600,1.894000,1.799800,1.799800,26950.980469,1510315199999,4.926688e+04,121,5299.279785,9.701802e+03,BNBUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-28 12:00:00,2023-10-28 12:00:00,226.800003,227.000000,224.800003,225.300003,26613.314453,1698497999999,6.011434e+06,9671,10624.792969,2.400106e+06,BNBUSDT
2023-10-28 13:00:00,2023-10-28 13:00:00,225.300003,226.899994,225.100006,226.899994,19924.443359,1698501599999,4.507830e+06,7855,11005.630859,2.490085e+06,BNBUSDT
2023-10-28 14:00:00,2023-10-28 14:00:00,226.899994,227.000000,225.899994,226.100006,19036.894531,1698505199999,4.311788e+06,7672,8968.831055,2.031645e+06,BNBUSDT


In [4]:
data = calc_utils.calc_RSI(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52161 entries, 2017-11-10 07:00:00 to 2023-10-28 16:00:00
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     52161 non-null  datetime64[ns]
 1   open                          52161 non-null  float32       
 2   high                          52161 non-null  float32       
 3   low                           52161 non-null  float32       
 4   close                         52161 non-null  float32       
 5   volume                        52161 non-null  float32       
 6   close_time                    52161 non-null  int64         
 7   quote_asset_volume            52161 non-null  float32       
 8   number_of_trades              52161 non-null  int16         
 9   taker_buy_base_asset_volume   52161 non-null  float32       
 10  taker_buy_quote_asset_volume  52161 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi
ix_open_time,,,,,,,,,,,,,
2017-11-10 07:00:00,2017-11-10 07:00:00,1.948700,1.948800,1.895000,1.895000,1751.890015,1510300799999,3.368215e+03,22,308.000000,6.002211e+02,BNBUSDT,NaN
2017-11-10 08:00:00,2017-11-10 08:00:00,1.945900,1.970000,1.880000,1.970000,43483.128906,1510304399999,8.331019e+04,95,18540.289062,3.608887e+04,BNBUSDT,NaN
2017-11-10 09:00:00,2017-11-10 09:00:00,1.930100,2.010100,1.850000,1.850000,37745.929688,1510307999999,7.401212e+04,123,14075.990234,2.809465e+04,BNBUSDT,NaN
2017-11-10 10:00:00,2017-11-10 10:00:00,1.833500,1.920000,1.833500,1.890000,9118.969727,1510311599999,1.681954e+04,35,1742.750000,3.280829e+03,BNBUSDT,NaN
2017-11-10 11:00:00,2017-11-10 11:00:00,1.855600,1.894000,1.799800,1.799800,26950.980469,1510315199999,4.926688e+04,121,5299.279785,9.701802e+03,BNBUSDT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-28 12:00:00,2023-10-28 12:00:00,226.800003,227.000000,224.800003,225.300003,26613.314453,1698497999999,6.011434e+06,9671,10624.792969,2.400106e+06,BNBUSDT,47.597000
2023-10-28 13:00:00,2023-10-28 13:00:00,225.300003,226.899994,225.100006,226.899994,19924.443359,1698501599999,4.507830e+06,7855,11005.630859,2.490085e+06,BNBUSDT,54.415493
2023-10-28 14:00:00,2023-10-28 14:00:00,226.899994,227.000000,225.899994,226.100006,19036.894531,1698505199999,4.311788e+06,7672,8968.831055,2.031645e+06,BNBUSDT,50.852654


In [5]:
data = calc_utils.calc_ema_periods(data, periods_of_time=[times_regression_PnL, 200])
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52161 entries, 2017-11-10 07:00:00 to 2023-10-28 16:00:00
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     52161 non-null  datetime64[ns]
 1   open                          52161 non-null  float32       
 2   high                          52161 non-null  float32       
 3   low                           52161 non-null  float32       
 4   close                         52161 non-null  float32       
 5   volume                        52161 non-null  float32       
 6   close_time                    52161 non-null  int64         
 7   quote_asset_volume            52161 non-null  float32       
 8   number_of_trades              52161 non-null  int16         
 9   taker_buy_base_asset_volume   52161 non-null  float32       
 10  taker_buy_quote_asset_volume  52161 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_24p,ema_24p_diff,ema_200p,ema_200p_diff
ix_open_time,,,,,,,,,,,,,,,,,
2017-11-10 07:00:00,2017-11-10 07:00:00,1.948700,1.948800,1.895000,1.895000,1751.890015,1510300799999,3.368215e+03,22,308.000000,6.002211e+02,BNBUSDT,NaN,NaN,NaN,NaN,NaN
2017-11-10 08:00:00,2017-11-10 08:00:00,1.945900,1.970000,1.880000,1.970000,43483.128906,1510304399999,8.331019e+04,95,18540.289062,3.608887e+04,BNBUSDT,NaN,NaN,NaN,NaN,NaN
2017-11-10 09:00:00,2017-11-10 09:00:00,1.930100,2.010100,1.850000,1.850000,37745.929688,1510307999999,7.401212e+04,123,14075.990234,2.809465e+04,BNBUSDT,NaN,NaN,NaN,NaN,NaN
2017-11-10 10:00:00,2017-11-10 10:00:00,1.833500,1.920000,1.833500,1.890000,9118.969727,1510311599999,1.681954e+04,35,1742.750000,3.280829e+03,BNBUSDT,NaN,NaN,NaN,NaN,NaN
2017-11-10 11:00:00,2017-11-10 11:00:00,1.855600,1.894000,1.799800,1.799800,26950.980469,1510315199999,4.926688e+04,121,5299.279785,9.701802e+03,BNBUSDT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-28 12:00:00,2023-10-28 12:00:00,226.800003,227.000000,224.800003,225.300003,26613.314453,1698497999999,6.011434e+06,9671,10624.792969,2.400106e+06,BNBUSDT,47.597000,225.910751,-0.270349,221.592682,1.673034
2023-10-28 13:00:00,2023-10-28 13:00:00,225.300003,226.899994,225.100006,226.899994,19924.443359,1698501599999,4.507830e+06,7855,11005.630859,2.490085e+06,BNBUSDT,54.415493,225.989883,0.402722,221.645493,2.370678
2023-10-28 14:00:00,2023-10-28 14:00:00,226.899994,227.000000,225.899994,226.100006,19036.894531,1698505199999,4.311788e+06,7672,8968.831055,2.031645e+06,BNBUSDT,50.852654,225.998703,0.044825,221.689819,1.989350


In [6]:
data = utils.regression_PnL(
    data=data,
    label=myenv.label,
    diff_percent=float(stop_loss),
    max_regression_profit_and_loss=int(times_regression_PnL),
    drop_na=True,
    drop_calc_cols=True,
    strategy=None)
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 51962 entries, 2017-11-18 14:00:00 to 2023-10-28 16:00:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     51962 non-null  datetime64[ns]
 1   open                          51962 non-null  float32       
 2   high                          51962 non-null  float32       
 3   low                           51962 non-null  float32       
 4   close                         51962 non-null  float32       
 5   volume                        51962 non-null  float32       
 6   close_time                    51962 non-null  int64         
 7   quote_asset_volume            51962 non-null  float32       
 8   number_of_trades              51962 non-null  int16         
 9   taker_buy_base_asset_volume   51962 non-null  float32       
 10  taker_buy_quote_asset_volume  51962 non-null  float32      

In [7]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 51962 entries, 2017-11-18 14:00:00 to 2023-10-28 16:00:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     51962 non-null  datetime64[ns]
 1   open                          51962 non-null  float32       
 2   high                          51962 non-null  float32       
 3   low                           51962 non-null  float32       
 4   close                         51962 non-null  float32       
 5   volume                        51962 non-null  float32       
 6   close_time                    51962 non-null  int64         
 7   quote_asset_volume            51962 non-null  float32       
 8   number_of_trades              51962 non-null  int16         
 9   taker_buy_base_asset_volume   51962 non-null  float32       
 10  taker_buy_quote_asset_volume  51962 non-null  float32      

In [8]:
perc_data_label = data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(data)
perc_data_label

,open_time,perc
status,,
CAI_1.5,21868,0.420846
ESTAVEL,7449,0.143355
SOBE_1.5,22645,0.435799


In [9]:
train_data, test_data = utils.split_train_test_data(data)
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 49801 entries, 2017-11-18 14:00:00 to 2023-07-30 15:00:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     49801 non-null  datetime64[ns]
 1   open                          49801 non-null  float32       
 2   high                          49801 non-null  float32       
 3   low                           49801 non-null  float32       
 4   close                         49801 non-null  float32       
 5   volume                        49801 non-null  float32       
 6   close_time                    49801 non-null  int64         
 7   quote_asset_volume            49801 non-null  float32       
 8   number_of_trades              49801 non-null  int16         
 9   taker_buy_base_asset_volume   49801 non-null  float32       
 10  taker_buy_quote_asset_volume  49801 non-null  float32      

In [10]:
perc_data_label = train_data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(train_data)
perc_data_label

,open_time,perc
status,,
CAI_1.5,21452,0.430754
ESTAVEL,6135,0.123190
SOBE_1.5,22214,0.446055


In [11]:
perc_data_label = test_data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(test_data)
perc_data_label

,open_time,perc
status,,
CAI_1.5,416,0.192503
ESTAVEL,1314,0.608052
SOBE_1.5,431,0.199445


In [12]:
# BTCUSDT 1h best params: close,volume,quote_asset_volume,number_of_trades,rsi
# text_numeric_features = 'volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'.split(',')
#text_numeric_features = 'close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'
#text_numeric_features = f'volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ema_{int(times_regression_PnL)}p'
#text_numeric_features = f'quote_asset_volume,number_of_trades,ema_{int(times_regression_PnL)}p,ema_200p'
text_numeric_features = f'close,number_of_trades,ema_24p'
#text_numeric_features2 = f'volume,taker_buy_base_asset_volume,ema_{int(times_regression_PnL)}p'

numeric_features = text_numeric_features.split(',')
print(f'Numeric Features: {numeric_features} - size: {len(numeric_features)}\n')

combination_numeric_features = []
if apply_combination_features:
	for size in range(1, len(numeric_features) + 1): 
		comb = map(list, combinations(numeric_features, size))	
		for c in comb:
			res = ''
			for j in c:
				res += f'{j},'
			combination_numeric_features.append(res[0:len(res) - 1])
else:
	combination_numeric_features = [text_numeric_features]

print(f'Combination Numeric Features: \n{combination_numeric_features}')

Numeric Features: ['close', 'number_of_trades', 'ema_24p'] - size: 3

Combination Numeric Features: 
['close,number_of_trades,ema_24p']


In [13]:
from datetime import datetime
import os
from imblearn.under_sampling import InstanceHardnessThreshold, RepeatedEditedNearestNeighbours, AllKNN


#iht = InstanceHardnessThreshold(estimator=KNeighborsClassifier())

# fix_imbalance_method: condensednearestneighbour, editednearestneighborus, repeatededitednearestneighbours, allknn, instancehardnessthreshold, nearmiss, neighbourhoodcleaningrule, onesidedselection, randomundersampler, tomeklinks, randomoversampler, smote, smotenc, smoten, adasyn, borderlinesmote, kmeanssmote, svmsmote, smoteenn, smotetomek.
# 'condensednearestneighbour,editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
# 'smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
#imbalance_list = 'editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
imbalance_list = ['instancehardnessthreshold']
simulation_results_filename = f'./resultado_simulacao_{symbol}_{interval}.csv'
print(f'simulation_results_filename: {simulation_results_filename}')

df_resultado_simulacao = pd.DataFrame()
has_simulation_file = os.path.exists(simulation_results_filename)
if has_simulation_file:
  df_resultado_simulacao = pd.read_csv(simulation_results_filename, sep=';')

setup = None

for aux_numeric_features in combination_numeric_features:
  experiement = ClassificationExperiment()
  features = []
  features += ['open_time', myenv.label]
  features += aux_numeric_features.split(',')
  print(f'features: {features}')
  for fix_imbalance_method in imbalance_list:


    print(f'fix_imbalance_method: {fix_imbalance_method}')
    fim = fix_imbalance_method
    match(fix_imbalance_method):
      case 'repeatededitednearestneighbours':
        fim = RepeatedEditedNearestNeighbours(kind_sel='all', max_iter=100, n_jobs=20, n_neighbors=3, sampling_strategy='auto')
      case 'instancehardnessthreshold':
        fim = InstanceHardnessThreshold(cv=5, estimator=None, n_jobs=20, random_state=123, sampling_strategy='auto')
      case 'allknn':
        fim = AllKNN(allow_minority=False, kind_sel='all', n_jobs=20, n_neighbors=3, sampling_strategy='auto')

    setup = experiement.setup(
        data=train_data[features].copy(),
        train_size=myenv.train_size,
        target=myenv.label,
        numeric_features=aux_numeric_features.split(','),
        date_features=['open_time'],
        create_date_columns=["hour", "day", "month"],
        data_split_shuffle=False,
        data_split_stratify=False,
        fix_imbalance=True,
        fix_imbalance_method=fim,
        remove_outliers=True,
        fold_strategy='timeseries', 
        fold=3,
        session_id=123,
        normalize=normalize,
        use_gpu=False,
        verbose=True,
        n_jobs=20,
        log_experiment=False,
    )

    best = setup.create_model(estimator, n_jobs=20, return_train_score=False, verbose=False)



simulation_results_filename: ./resultado_simulacao_BNBUSDT_1h.csv
features: ['open_time', 'status', 'close', 'number_of_trades', 'ema_24p']
fix_imbalance_method: instancehardnessthreshold


,Description,Value
0,Session id,123
1,Target,status
2,Target type,Multiclass
3,Target mapping,"CAI_1.5: 0, ESTAVEL: 1, SOBE_1.5: 2"
4,Original data shape,"(49801, 5)"
5,Transformed data shape,"(23129, 7)"
6,Transformed train set shape,"(8188, 7)"
7,Transformed test set shape,"(14941, 7)"
8,Numeric features,3
9,Date features,1


In [14]:
if tune_model:
	best = setup.tune_model(best)  



,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.4927,0.5401,0.4927,0.4665,0.4517,0.0540,0.0615
1,0.4333,0.5131,0.4333,0.4226,0.4198,0.0190,0.0195
2,0.4567,0.5062,0.4567,0.4796,0.4503,0.0108,0.0115
Mean,0.4609,0.5198,0.4609,0.4562,0.4406,0.0279,0.0308
Std,0.0244,0.0146,0.0244,0.0244,0.0147,0.0188,0.0219


Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [15]:
final_model = setup.finalize_model(best, model_only=True) 
final_model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=20, n_neighbors=50, p=2,
                     weights='uniform')

In [16]:
ajusted_test_data = test_data[features].drop(columns=[myenv.label])
ajusted_test_data.info()

df_final_predict = setup.predict_model(final_model, data=ajusted_test_data, raw_score=False, verbose=False)

res_score = None
if test_data is not None:
	df_final_predict[myenv.label] = test_data[myenv.label]
	df_final_predict['_score'] = df_final_predict['prediction_label'] == df_final_predict[myenv.label]

	print(f'Score Mean: {df_final_predict["_score"].mean()}')
	print(f'Score Group: \n{df_final_predict[[myenv.label, "_score"]].groupby(myenv.label).mean()}')
	res_score = df_final_predict[[myenv.label, '_score']].groupby(myenv.label).mean().copy()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2161 entries, 2023-07-30 16:00:00 to 2023-10-28 16:00:00
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   open_time         2161 non-null   datetime64[ns]
 1   close             2161 non-null   float32       
 2   number_of_trades  2161 non-null   int16         
 3   ema_24p           2161 non-null   float32       
dtypes: datetime64[ns](1), float32(2), int16(1)
memory usage: 54.9 KB
Score Mean: 0.4211013419713096
Score Group: 
            _score
status            
CAI_1.5   0.502404
ESTAVEL   0.515221
SOBE_1.5  0.055684
